In [1]:
import os
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Generic data loading function
def load_data(file, header=False):
    csv_path = os.path.join(file)
    return pd.read_csv(csv_path)

In [3]:
features_pd = load_data("features.csv")
labels_pd = load_data("labels.csv")
digits_pd = load_data("digits.csv")

In [4]:
features_pd.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.936102,0.715722,0.890466,0.390709,0.282481,0.253156,0.233479,0.215873,0.229388,0.244324,...,0.207214,0.24740,0.217562,0.134784,0.110725,0.079305,0.0,0.0,0.0,0.0
1,-3.154901,-2.776462,-2.567235,-3.494515,-4.016681,-4.725573,-5.652502,-7.023488,-5.427537,-3.503317,...,-2.401020,-4.82666,-4.505860,0.134784,0.110725,0.079305,0.0,0.0,0.0,0.0
2,0.630462,0.520364,0.890466,0.390709,0.282481,0.253156,0.233479,0.215873,0.229388,0.244324,...,0.207214,0.24740,0.217562,0.134784,0.110725,0.079305,0.0,0.0,0.0,0.0
3,0.458512,0.723576,0.297072,-0.753322,0.282481,0.253156,0.233479,0.215873,0.229388,0.244324,...,0.207214,0.24740,0.217562,0.134784,0.110725,0.079305,0.0,0.0,0.0,0.0
4,-1.086478,-1.003072,-1.338621,0.390709,0.282481,0.253156,0.233479,0.215873,0.229388,0.244324,...,0.207214,0.24740,0.217562,0.134784,0.110725,0.079305,0.0,0.0,0.0,0.0


In [5]:
labels_pd.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
digits_pd.head()

,0
0,5
1,2
2,1
3,1
4,0


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit

def strat_shuffle_split(features, labels, n_splits=1, test_size=0.2, random_state=42):
    split = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=random_state)
    for train_index, test_index in split.split(digits_pd, digits_pd["0"]):
        train_features = features[train_index]
        test_features = features[test_index]
        train_labels = labels[train_index]
        test_labels = labels[test_index]
        return (train_features, train_labels), (test_features, test_labels)

    return None

In [8]:
features = features_pd.values
features = np.reshape(features, (len(features), 20, 25 ,1))
labels = labels_pd.values

In [9]:
train_data, test_data = strat_shuffle_split(features, labels)

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping

x_train = train_data[0]
y_train = train_data[1]
x_test = test_data[0]
y_test = test_data[1]

model = Sequential()
model.add(Conv2D(128, kernel_size=(3, 3), strides=(1,1), activation='elu', padding='same', input_shape=(20, 25, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(64, kernel_size=(2, 2), strides=(1,1), activation='elu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
          
model.add(Conv2D(64, kernel_size=(2, 2), strides=(1,1), activation='elu', padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
          
model.add(Flatten())
          
model.add(Dense(256, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
          
model.add(Dense(128, activation='elu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
          
model.add(Dense(40, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

          
custom_callbacks = [ModelCheckpoint("model.h5", monitor='val_acc', verbose=1, save_best_only=True, 
                             save_weights_only=False, mode='max', period=1)]

          
model.fit(x_train, y_train, 
          batch_size=10, 
          epochs=30, 
          verbose=1, 
          callbacks=custom_callbacks,
          validation_data=(x_test, y_test))

Using TensorFlow backend.


Train on 1280 samples, validate on 320 samples
Epoch 1/30
1280/1280 [==============================] - 35s 27ms/step - loss: 1.9145 - acc: 0.5102 - val_loss: 0.8275 - val_acc: 0.7875

Epoch 00001: val_acc improved from -inf to 0.78750, saving model to model.h5
Epoch 2/30
1280/1280 [==============================] - 33s 26ms/step - loss: 0.8993 - acc: 0.7492 - val_loss: 0.5279 - val_acc: 0.8625

Epoch 00002: val_acc improved from 0.78750 to 0.86250, saving model to model.h5
Epoch 3/30
1280/1280 [==============================] - 33s 26ms/step - loss: 0.6052 - acc: 0.8367 - val_loss: 0.3763 - val_acc: 0.8875

Epoch 00003: val_acc improved from 0.86250 to 0.88750, saving model to model.h5
Epoch 4/30
1280/1280 [==============================] - 30s 23ms/step - loss: 0.4719 - acc: 0.8672 - val_loss: 0.3447 - val_acc: 0.8906

Epoch 00004: val_acc improved from 0.88750 to 0.89062, saving model to model.h5
Epoch 5/30
1280/1280 [==============================] - 28s 22ms/step - loss: 0.3808 - a

In [11]:
from keras.models import load_model
model = None
model = load_model("model.h5")

In [12]:
test_eval = model.evaluate(x_test, y_test, verbose=1)
print("\nTest eval:", "\nLoss: ", test_eval[0], "\nAccuracy: ", test_eval[1])

320/320 [==============================] - 1s 2ms/step

Test eval: 
Loss:  0.15323573686182498 
Accuracy:  0.9625


In [13]:
total_eval = model.evaluate(features, labels, verbose=1)
print("\nTotal eval:", "\nLoss: ", total_eval[0], "\nAccuracy: ", total_eval[1])

1600/1600 [==============================] - 2s 2ms/step

Total eval: 
Loss:  0.03991389439906925 
Accuracy:  0.99125


In [14]:
print("Details about wrong predictions")
preds = model.predict(features)
size = len(preds)
files_pd = load_data("files.csv")
files = files_pd.values

for idx in range(size):
    pred = np.argmax(preds[idx])
    label = np.argmax(labels[idx])
    if pred == label: continue
    print("\n\nID: ", idx, " Name: ", files[idx])
    print("Preds: ", preds[idx])
    print("Actual Person: ", int(label / 10))
    print("Predicted Person: ", int(pred / 10))
    print("Actual Digit: ", (label % 10))
    print("Predicted Digit: ", (pred % 10))

Details about wrong predictions


ID:  21  Name:  ['./recordings/9_theo_45.wav']
Preds:  [1.95788689e-05 6.98053191e-05 1.99647238e-05 1.17380827e-04
 1.08685745e-05 2.64667498e-04 1.90586041e-04 3.53783980e-04
 4.22902987e-04 5.50303515e-03 3.37768847e-06 2.15531350e-03
 1.48915173e-03 3.23070941e-04 3.45097687e-05 1.86928036e-03
 5.24630705e-05 7.16999330e-06 1.03679246e-04 8.44124224e-05
 1.07093319e-05 1.82432592e-01 1.85961660e-03 1.67108548e-04
 5.87697541e-05 6.39015019e-01 1.22107577e-03 1.02587219e-03
 9.89707314e-06 1.58138350e-01 3.22901848e-04 7.09535379e-04
 6.67487911e-05 3.06451315e-04 5.24730829e-04 1.95700690e-04
 2.59286637e-04 5.82242719e-05 4.31851688e-04 9.05984562e-05]
Actual Person:  2
Predicted Person:  2
Actual Digit:  9
Predicted Digit:  5


ID:  316  Name:  ['./recordings/1_nicolas_12.wav']
Preds:  [1.2444616e-04 5.7128194e-04 1.6810225e-04 2.5845042e-03 6.8374799e-04
 4.4728472e-04 6.5023766e-04 4.5071862e-04 2.1965036e-04 1.4988434e-03
 4.3748165e-04 1.3021